# Install Java

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz

# Set Environment Variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"
os.environ["PYTHONPATH"] = "/content/spark-3.5.3-bin-hadoop3/python"

# Install PySpark

In [ ]:
!pip install findspark
import findspark
findspark.init()

# Install Apache Sedona

In [ ]:
!pip install apache-sedona[spark]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 11.5 MB/s eta 0:00:00


# Start Sedona

In [ ]:
from sedona.spark import *
config = SedonaContext.builder(). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.0_2.12:1.6.1,'
           'org.datasyslab:geotools-wrapper:1.6.1-28.2'). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    getOrCreate()
sedona = SedonaContext.create(config)

# Example Reading Data
Using earthquake data from https://www.kaggle.com/datasets/warcoder/earthquake-dataset?select=earthquake_data.csv

In [ ]:
# RDD (Resilient Distributed Database) approach
# more information here: https://sedona.apache.org/1.5.1/tutorial/rdd/
# First modify the file to be longitude, latitude, time format
df = sedona.read.csv('earthquake_data.csv', header=True, inferSchema=True)
df = df[["latitude", "longitude", "date_time"]]
df.write.csv("output", header=False)
# the output is a directory, with part 1 file that contains the data

In [ ]:
all_records[0:5]

[Geometry: Point userData: 22-11-2022 02:03,
 Geometry: Point userData: 18-11-2022 13:37,
 Geometry: Point userData: 12-11-2022 07:09,
 Geometry: Point userData: 11-11-2022 10:48,
 Geometry: Point userData: 09-11-2022 10:14]

Own code here onwards :

In [ ]:
import json
import pandas as pd

#Load dataset
file_path = '/2017-07-22_09-02-53.txt'

# Step 2: Read file & parse each JSON object
json_data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        #parse each line as a JSON object
        try:
            data = json.loads(line)
            json_data.append(data)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line.strip()}\nError: {e}")

#Convert the list of JSON objects into a DataFrame
df = pd.DataFrame(json_data)

# Step 4: Display dataset schema
print("Schema of the Data:")
print(df.dtypes)

# Display a sample of the data
print("\nSample Data:")
print(df.head())


Schema of the Data:
created_at                    object
id                           float64
id_str                        object
text                          object
display_text_range            object
source                        object
truncated                     object
in_reply_to_status_id        float64
in_reply_to_status_id_str     object
in_reply_to_user_id          float64
in_reply_to_user_id_str       object
in_reply_to_screen_name       object
user                          object
geo                           object
coordinates                   object
place                         object
contributors                 float64
is_quote_status               object
retweet_count                float64
favorite_count               float64
entities                      object
favorited                     object
retweeted                     object
filter_level                  object
lang                          object
timestamp_ms                  object
extended_entities 

Py4JJavaError: An error occurred while calling o79.text.
: java.lang.NoClassDefFoundError: org/locationtech/jts/geom/Geometry
	at org.apache.spark.sql.sedona_sql.UDT.UdtRegistratorWrapper$.registerAll(UdtRegistratorWrapper.scala:27)
	at org.apache.sedona.sql.UDT.UdtRegistrator$.registerAll(UdtRegistrator.scala:26)
	at org.apache.sedona.sql.utils.SedonaSQLRegistrator$.registerAll(SedonaSQLRegistrator.scala:33)
	at org.apache.sedona.sql.SedonaSqlExtensions.$anonfun$apply$1(SedonaSqlExtensions.scala:28)
	at org.apache.spark.sql.SparkSessionExtensions.$anonfun$buildCheckRules$1(SparkSessionExtensions.scala:249)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.co

In [ ]:
#Initialization and Data Loading

import time
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator

# Initialize Spark session and Sedona
spark = SparkSession.builder \
    .appName("SpatioTemporalQuery") \
    .getOrCreate()

# Register Sedona functions
SedonaRegistrator.registerAll(spark)

# Load the dataset (the data is in JSON format)
file_path = "/content/2017-07-22_09-02-53.txt"
df = spark.read.json(file_path)

# Show the schema to verify the structure of the data
df.printSchema()



<ipython-input-17-16f21ff21fa0>:13: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [ ]:
!pip install apache-sedona
#Data Preparation

import time
from pyspark.sql import SparkSession, functions as F
# Import Sedona functions correctly
from sedona.register import SedonaRegistrator
from pyspark.sql.functions import expr

# Initialize Spark session and Sedona
spark = SparkSession.builder \
    .appName("SpatioTemporalQuery") \
    .getOrCreate()

# Register Sedona functions
SedonaRegistrator.registerAll(spark)


# Parse time as a timestamp and coordinates as double
df = df.withColumn(
    "timestamp", F.col("created_at").cast("timestamp")
)

# Extract latitude
df = df.withColumn(
    "latitude",
    F.when(
        F.col("coordinates").isNull() & F.col("place").isNotNull(),
        F.expr("""
            aggregate(
                transform(place.bounding_box.coordinates[0], point -> cast(point[1] as double)),
                0.0D,
                (total, current) -> total + current
            ) / 4
        """)
    ).otherwise(F.col("coordinates").getField("coordinates")[1])
)

# Extract longitude
df = df.withColumn(
    "longitude",
    F.when(
        F.col("coordinates").isNull() & F.col("place").isNotNull(),
        F.expr("""
            aggregate(
                transform(place.bounding_box.coordinates[0], point -> cast(point[0] as double)),
                0.0D,
                (total, current) -> total + current
            ) / 4
        """)
    ).otherwise(F.col("coordinates").getField("coordinates")[0])
)

# Create a point column using expr and ST_Point (Sedona function)
df = df.withColumn("location", expr("ST_Point(longitude, latitude)"))

# Show a sample of the data to ensure correctness
df.select("created_at", "latitude", "longitude", "location").show(5, truncate=False)

<ipython-input-19-c6c3031de4a1>:16: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


+------------------------------+------------------+-----------+--------------------------------------+
|created_at                    |latitude          |longitude  |location                              |
+------------------------------+------------------+-----------+--------------------------------------+
|Sat Jul 22 09:02:53 +0000 2017|53.4569525        |-2.23348   |POINT (-2.23348 53.4569525)           |
|Sat Jul 22 09:02:53 +0000 2017|28.377247         |129.493744 |POINT (129.493744 28.377247)          |
|Sat Jul 22 09:02:53 +0000 2017|10.496            |-66.8983   |POINT (-66.8983 10.496)               |
|Sat Jul 22 09:02:52 +0000 2017|22.869936000000003|113.4197245|POINT (113.4197245 22.869936000000003)|
|Sat Jul 22 09:02:53 +0000 2017|35.701657         |139.7091805|POINT (139.7091805 35.701657)         |
+------------------------------+------------------+-----------+--------------------------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, lit
from sedona.register import SedonaRegistrator
# Directly importing the functions
from sedona.sql import ST_Point, ST_Within, ST_GeomFromText

# Initialize Spark session and Sedona
spark = SparkSession.builder \
    .appName("SpatialTemporalQuery") \
    .getOrCreate()

# Register Sedona UDFs
SedonaRegistrator.registerAll(spark)

# Load the dataset (replace with actual path)
df = spark.read.json("/content/2017-07-22_09-02-53.txt")

# Extract latitude and longitude based on the presence of 'coordinates'
df = df.withColumn(
    "latitude",
    when(
        col("coordinates").isNull() & col("place").isNotNull(),
        expr("""
            aggregate(
                transform(place.bounding_box.coordinates[0], point -> cast(point[1] as double)),
                0.0D,
                (total, current) -> total + current
            ) / 4
        """)
    ).otherwise(
        col("coordinates").getField("coordinates")[1]
    )
)

df = df.withColumn(
    "longitude",
    when(
        col("coordinates").isNull() & col("place").isNotNull(),
        expr("""
            aggregate(
                transform(place.bounding_box.coordinates[0], point -> cast(point[0] as double)),
                0.0D,
                (total, current) -> total + current
            ) / 4
        """)
    ).otherwise(
        col("coordinates").getField("coordinates")[0]
    )
)

# Create a 'location' column as a spatial point
df = df.withColumn("location", ST_Point(col("longitude"), col("latitude")))

# Show the first few rows of the dataframe
df.select("latitude", "longitude", "location").show(5, truncate=False)

# Define your time range and bounding box (xmin, ymin, xmax, ymax)
t1 = "2017-01-01 00:00:00"
t2 = "2017-12-31 23:59:59"
xmin, ymin, xmax, ymax = 41.0, 12.0, 45.0, 20.0  # Example bounding box coordinates

# Convert 'created_at' to timestamp
df = df.withColumn("timestamp", unix_timestamp("created_at", "EEE MMM dd HH:mm:ss +0000 yyyy"))

# Spatial filtering: ST_Within is used for the spatial filter, and ST_GeomFromText to define the polygon
df_filtered = df.filter(
    (col("timestamp").between(unix_timestamp(lit(t1)), unix_timestamp(lit(t2)))) &
    ST_Within(col("location"), ST_GeomFromText(f"POLYGON(({xmin} {ymin}, {xmax} {ymin}, {xmax} {ymax}, {xmin} {ymax}, {xmin} {ymin}))"))
)

# Show the filtered results
df_filtered.show(5, truncate=False)


<ipython-input-29-ee114c60a065>:13: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


+------------------+-----------+--------------------------------------+
|latitude          |longitude  |location                              |
+------------------+-----------+--------------------------------------+
|53.4569525        |-2.23348   |POINT (-2.23348 53.4569525)           |
|28.377247         |129.493744 |POINT (129.493744 28.377247)          |
|10.496            |-66.8983   |POINT (-66.8983 10.496)               |
|22.869936000000003|113.4197245|POINT (113.4197245 22.869936000000003)|
|35.701657         |139.7091805|POINT (139.7091805 35.701657)         |
+------------------+-----------+--------------------------------------+
only showing top 5 rows



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `POLYGON((41`.`0 12`.`0, 45`.`0 12`.`0, 45`.`0 20`.`0, 41`.`0 20`.`0, 41`.`0 12`.`0))` cannot be resolved. Did you mean one of the following? [`contributors`, `coordinates`, `created_at`, `display_text_range`, `entities`].;
'Filter (((timestamp#796L >= unix_timestamp(2017-01-01 00:00:00, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC), false)) AND (timestamp#796L <= unix_timestamp(2017-12-31 23:59:59, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC), false))) AND  **org.apache.spark.sql.sedona_sql.expressions.ST_Within**  )
+- Project [contributors#593, coordinates#594, created_at#595, display_text_range#596, entities#597, extended_entities#598, extended_tweet#599, favorite_count#600L, favorited#601, filter_level#602, geo#603, id#604L, id_str#605, in_reply_to_screen_name#606, in_reply_to_status_id#607L, in_reply_to_status_id_str#608, in_reply_to_user_id#609L, in_reply_to_user_id_str#610, is_quote_status#611, lang#612,

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#from sedona.sql import functions as st
from sedona.sql import ST_GeomFromWKT, ST_Within
from shapely.geometry import Point

# Initialize Spark Session and SedonaRegistrator
spark = SparkSession.builder \
    .appName("GeoDataFiltering") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Register Sedona functions to Spark SQL
from sedona.register import SedonaRegistrator
SedonaRegistrator.registerAll(spark)

# Load dataset
df = spark.read.option("header", "true").csv("/content/2017-07-22_09-02-53.txt")

# Show schema
df.printSchema()

# Define a point to filter by
reference_point = Point(-74.0060, 40.7128)

# Register the geometry column as a WKT (Well-Known Text) format for Sedona
df = df.withColumn("geometry", ST_GeomFromWKT(col("wkt_column_name")))

# Filter data based on geospatial criteria - In this case, we check if the geometry is within a certain boundary.
boundary_wkt = "POLYGON((-74.0479 40.6839, -73.9106 40.6839, -73.9106 40.8776, -74.0479 40.8776, -74.0479 40.6839))"  # Example boundary polygon (NYC)

# Convert boundary to geometry
boundary_geometry = ST_GeomFromWKT(boundary_wkt)

# Apply the filter based on the geospatial condition
df_filtered = df.filter(ST_Within(col("geometry"), boundary_geometry))

# Show the first few filtered records
df_filtered.show(5, truncate=False)

<ipython-input-42-5dc575d9ee3d>:15: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


root
 |-- {"created_at":"Sat Jul 22 09:02:53 +0000 2017": string (nullable = true)
 |-- "id":888685771020009472: string (nullable = true)
 |-- "id_str":"888685771020009472": string (nullable = true)
 |-- "text":"@AmandaMullen3 @scottclarke948 Nothing but nothing beats feeling proud \u2764\u2764 I'm made up for you all": string (nullable = true)
 |-- "display_text_range":[31: string (nullable = true)
 |-- 97]: string (nullable = true)
 |-- "source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e": string (nullable = true)
 |-- "truncated":false: string (nullable = true)
 |-- "in_reply_to_status_id":888680020197199876: string (nullable = true)
 |-- "in_reply_to_status_id_str":"888680020197199876": string (nullable = true)
 |-- "in_reply_to_user_id":548470963: string (nullable = true)
 |-- "in_reply_to_user_id_str":"548470963": string (nullable = true)
 |-- "in_reply_to_screen_name":"AmandaMullen3": string (nullable = true

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `wkt_column_name` cannot be resolved. Did you mean one of the following? [` England"`, `"geo":null`, `"url":null`, `"retweet_count":0`, `"symbols":[]}`].;
'Project [{"created_at":"Sat Jul 22 09:02:53 +0000 2017"#1614, "id":888685771020009472#1615, "id_str":"888685771020009472"#1616, "text":"@AmandaMullen3 @scottclarke948 Nothing but nothing beats feeling proud \u2764\u2764 I'm made up for you all"#1617, "display_text_range":[31#1618, 97]#1619, "source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e"#1620, "truncated":false#1621, "in_reply_to_status_id":888680020197199876#1622, "in_reply_to_status_id_str":"888680020197199876"#1623, "in_reply_to_user_id":548470963#1624, "in_reply_to_user_id_str":"548470963"#1625, "in_reply_to_screen_name":"AmandaMullen3"#1626, "user":{"id":581151219#1627, "id_str":"581151219"#1628, "name":"I A

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from sedona.sql import ST_GeomFromWKT, ST_Within #Updated imports
# Import for creating the spatial RDD
from sedona.core.SpatialRDD import SpatialRDD
from shapely.geometry import Point

# Initialize Spark Session and SedonaRegistrator
spark = SparkSession.builder \
    .appName("GeoDataFiltering") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Register Sedona functions to Spark SQL
from sedona.register import SedonaRegistrator
SedonaRegistrator.registerAll(spark)

# Load your dataset
df = spark.read.option("header", "true").csv("/content/2017-07-22_09-02-53.txt")

# Show schema
df.printSchema()

# Define a point to filter by
reference_point = Point(-74.0060, 40.7128)

#Let 'wkt_column_name' is the column containing WKT data
df = df.withColumn("geometry", ST_GeomFromWKT(col("wkt_column_name")))

# Create a Sedona spatial RDD from the DataFrame
point_rdd = SpatialRDD.fromDataFrame(df, geometryFieldName="geometry")

# Perform Spatial Partitioning
# This can be done by setting the spatial partitioner for the RDD (e.g., using a grid partitioner)
point_rdd.spatialPartitioning("grid")

# And, you can perform range or nearest neighbor queries using the R-Tree index
point_rdd.buildIndex("RTREE", True)

# Now, the data's spatially partitioned and indexed using an R-tree

# Define a query geometry (for example, a bounding box or another region)
query_wkt = "POLYGON((-74.0479 40.6839, -73.9106 40.6839, -73.9106 40.8776, -74.0479 40.8776, -74.0479 40.6839))"  # Example NYC boundary

# Convert the query geometry to R-tree format
query_geometry = st.ST_GeomFromWKT(query_wkt)

# Perform spatial filter to find data within the query geometry (using R-Tree index for optimization)
filtered_df = df.filter(st.ST_Within(col("geometry"), query_geometry))

# Show the first few filtered records
filtered_df.show(5, truncate=False)

#Use the R-tree index for spatial query, via the RDD
result_rdd = point_rdd.rangeQuery(query_geometry, True)

# Convert the result back to a DataFrame
result_df = spark.createDataFrame(result_rdd.collect(), ["geometry"])
result_df.show(5, truncate=False)


<ipython-input-44-48263306604a>:17: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


root
 |-- {"created_at":"Sat Jul 22 09:02:53 +0000 2017": string (nullable = true)
 |-- "id":888685771020009472: string (nullable = true)
 |-- "id_str":"888685771020009472": string (nullable = true)
 |-- "text":"@AmandaMullen3 @scottclarke948 Nothing but nothing beats feeling proud \u2764\u2764 I'm made up for you all": string (nullable = true)
 |-- "display_text_range":[31: string (nullable = true)
 |-- 97]: string (nullable = true)
 |-- "source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e": string (nullable = true)
 |-- "truncated":false: string (nullable = true)
 |-- "in_reply_to_status_id":888680020197199876: string (nullable = true)
 |-- "in_reply_to_status_id_str":"888680020197199876": string (nullable = true)
 |-- "in_reply_to_user_id":548470963: string (nullable = true)
 |-- "in_reply_to_user_id_str":"548470963": string (nullable = true)
 |-- "in_reply_to_screen_name":"AmandaMullen3": string (nullable = true

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `wkt_column_name` cannot be resolved. Did you mean one of the following? [` England"`, `"geo":null`, `"url":null`, `"retweet_count":0`, `"symbols":[]}`].;
'Project [{"created_at":"Sat Jul 22 09:02:53 +0000 2017"#1836, "id":888685771020009472#1837, "id_str":"888685771020009472"#1838, "text":"@AmandaMullen3 @scottclarke948 Nothing but nothing beats feeling proud \u2764\u2764 I'm made up for you all"#1839, "display_text_range":[31#1840, 97]#1841, "source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e"#1842, "truncated":false#1843, "in_reply_to_status_id":888680020197199876#1844, "in_reply_to_status_id_str":"888680020197199876"#1845, "in_reply_to_user_id":548470963#1846, "in_reply_to_user_id_str":"548470963"#1847, "in_reply_to_screen_name":"AmandaMullen3"#1848, "user":{"id":581151219#1849, "id_str":"581151219"#1850, "name":"I A

**References :**

Slides and research papers covered in the lectures

https://users.cs.utah.edu/~lifeifei/cis5930/lecture3.pdf

https://ieeexplore.ieee.org/document/5567999

https://www.youtube.com/watch?v=G3UomcjQXkk&list=PLyWyQBSWLw1OF3RGVWToPkqe_pfx4TcPW

https://www.geeksforgeeks.org/introduction-to-r-tree/